In [206]:
import re
import sys
import requests

import gensim
import gensim.downloader as api
from gensim.models import KeyedVectors

import pandas as pd

from tqdm import tqdm

from pymystem3 import Mystem

from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split

### Импортируем датасет и предобученную модель русского языка

In [207]:
%%time
dataframe = pd.read_excel('/Users/magomednikolaev/Documents/Работа/NLP/we/comments.xlsx')
russian_model = gensim.models.KeyedVectors.load_word2vec_format('/Users/magomednikolaev/Documents/Работа/NLP/we/we/186/model.txt', binary=False)

CPU times: user 53.3 s, sys: 2.01 s, total: 55.3 s
Wall time: 59.8 s


### Обучим модель на уже имеющихся данных 

Удалим не имеющие отношения к делу атрибуты

In [208]:
del dataframe['text_init']
del dataframe['text_init_TOK']
del dataframe['bast_list']
del dataframe['word_cnt']

dataframe = dataframe[['text_TOK_str', 'rating_cat', 'is_informative']]
dataframe = dataframe.rename(columns = {'text_TOK_str' : 'text',
                          'rating_cat' : 'tone',
                          'is_informative' : 'informative'})

Пока возьмем только тональность 

In [209]:
dataframeTone = dataframe.drop({'informative'}, axis = 1) 
dataframeInf = dataframe.drop({'tone'}, axis = 1)

In [210]:
dataframeTone_train, dataframeTone_test = train_test_split(dataframeTone, test_size = 0.2, random_state = 42)

Будем использовать tf-idf vectorizer, ранее эта модель показала наилучшие параметры при ngram_range=(1, 4), поэтому пока остаивим этот параметр 

In [211]:
text_transformer = TfidfVectorizer(
                                   ngram_range=(1, 4), 
                                   lowercase=True, max_features=150000)

In [212]:
%%time
X_train = text_transformer.fit_transform(dataframeTone_train['text'])
X_test = text_transformer.transform(dataframeTone_test['text'])
y_train = dataframeTone_train['tone']
y_test = dataframeTone_test['tone']

CPU times: user 1.06 s, sys: 60 ms, total: 1.12 s
Wall time: 1.13 s


In [213]:
X_train.shape, X_test.shape

((4176, 150000), (1045, 150000))

В качестве модели возьмем логистическую регрессию

In [214]:
LogReg = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)
LogReg.fit(X_train, y_train)
y_pred = LogReg.predict(X_test)

In [215]:
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.85      0.92      0.88       496
     class 1       0.89      0.82      0.85       403
     class 2       0.88      0.79      0.83       146

    accuracy                           0.87      1045
   macro avg       0.87      0.85      0.86      1045
weighted avg       0.87      0.87      0.86      1045



### Теперь попробуем аугментировать данные и посмотрим на качество модели после этого

Напишем функцию(сопрем с гитхаба: https://github.com/akutuzov/webvectors/blob/master/preprocessing/rus_preprocessing_mystem.py), которая будет переводить mystem tags в upos tags, которые используются в нужных моделях. Несуществующие слова просто пропускаются, также, как и знаки препинания. 

In [216]:
def tag_mystem(
    text="Текст нужно передать функции в виде строки!", mapping=None, postags=True
):
    # если частеречные тэги не нужны (например, их нет в модели), выставьте postags=False
    # в этом случае на выход будут поданы только леммы

    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(",")[0]
            pos = pos.split("=")[0].strip()
            if mapping:
                if pos in mapping:
                    if pos == 'STOP':
                        break
                    pos = mapping[pos]  # здесь мы конвертируем тэги
                else:
                    pos = "X"  # на случай, если попадется тэг, которого нет в маппинге
            tagged.append(lemma.lower() + "_" + pos)
        except KeyError:
            continue  # я здесь пропускаю знаки препинания, но вы можете поступить по-другому
        except IndexError:
            continue # я здесь пропускаю слова с опечатками и просто несуществующие
            
    if not postags:
        tagged = [t.split("_")[0] for t in tagged]
    return tagged


# Таблица преобразования частеречных тэгов Mystem в тэги UPoS:
mapping_url = "https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map"

mystem2upos = {}
r = requests.get(mapping_url, stream=True)
for pair in r.text.split("\n"):
    pair = pair.split()
    if len(pair) > 1:
        mystem2upos[pair[0]] = pair[1]

print("Loading the model...", file=sys.stderr)
m = Mystem()

print("Processing input...", file=sys.stderr)
for line in sys.stdin:
    res = line.strip()
    output = tag_mystem(text=res, mapping=mystem2upos)
    print(" ".join(output))

Loading the model...
Processing input...


Посмотрим на возможные тэги

In [217]:
mystem2upos.values()

dict_values(['ADJ', 'ADV', 'ADV', 'ADJ', 'DET', 'ADJ', 'SCONJ', 'INTJ', 'X', 'NUM', 'PART', 'ADP', 'NOUN', 'PRON', 'X', 'VERB'])

Присвоим тэг всему тексту, каждому слову

In [218]:
# Пропускаем этот шаг, так как разбиение на выборки мы сдлеали заранее. 

# dataframeAug = dataframe.copy()
# dataframeAug.text = dataframeAug.text.apply(lambda x: tag_mystem(x, mapping = mystem2upos))

In [219]:
dataframeTrainAug = dataframeTone_train.copy()
dataframeTrainAug.text = dataframeTrainAug.text.apply(lambda x: tag_mystem(x, mapping = mystem2upos))

Посмотрим для начала, с какими проблемами может столкунться модель:
   1. Уже на этапе тэга не все слова могут найтись
   2. Даже по правильным тэгам не может найти 
   3. После применения замены появпялются "-"
   4. Слова с опечатками не проходят на одном из этапов: на моменте тэга(но может и пройти), на моменте поиска синонимичного слова(там уже при наличии ошибкии тэга, скорее всего ничего не сработает)

Саугментируем тренировочный корпус, соеденим два тренировочных датасета, а скор оценим на тестовом датасете

1. X_train is dataframeTone_train['text']
2. X_test = is dataframeTone_test['text']
3. y_train = dataframeTone_train['tone']
4. y_test = dataframeTone_test['tone']



In [220]:
def replacement_by_WE(pandas_Series):
    
    new_corpus = list()

    for text in tqdm(pandas_Series):

        new_text = list()
        for word in text:
            try:
                new_word = russian_model.most_similar(word)[0][0].split('_')[0]
                new_word = re.sub('-', '', new_word)
                new_word = re.sub(',', '', new_word)
            except KeyError:
                new_word = word.split('_')[0]

            new_text.append(new_word)
        new_corpus.append(' '.join(new_text))
        
    return new_corpus
    
    
dataframeTrainAug['text'] = replacement_by_WE(dataframeTrainAug.text) 

 15%|█▌        | 631/4176 [04:47<19:15,  3.07it/s]  /Users/magomednikolaev/opt/anaconda3/envs/nlp/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
100%|██████████| 4176/4176 [27:55<00:00,  2.49it/s]  


In [221]:
"Стало:", dataframeTrainAug.iloc[4000][0], "Было:", dataframeTone_train.iloc[4000][0]

('Стало:',
 'сделаюотреть регулировка роль быстро простой при огромный колличеств окошко жуткий тормоза',
 'Было:',
 'сделать настройка роль быстро просто при большой количество окно ужасный тормоз')

Скачаю аугментированный датасет,чтобы заново не обучать модель

In [222]:
dataframeTrainAug.to_csv("/Users/magomednikolaev/Documents/Работа/NLP/we/comments_train_augmeted_all_tags.xlsx",)

Чепуха:
  1. ,д
  2. ,з
  3. .таки
  4. ...ть
  5. б;льший

In [223]:
# Пропускаем этот шаг, так как разбиение на выборки мы сдлеали заранее. 

# dataframe_concated = pd.concat([dataframeTrainAug, dataframeTone_train])
# dataframe_concated = dataframe_concated.drop_duplicates(subset=['text'])

### Переобучим модель на аугментированных данных 

In [224]:
# Пропускаем этот шаг, так как разбиение на выборки мы сдлеали заранее. 

# dataframe_concatedTone = dataframe_concated.drop({'informative'}, axis = 1) 
# dataframe_concatedInf = dataframe_concated.drop({'tone'}, axis = 1)

In [225]:
# Пропускаем этот шаг, так как разбиение на выборки мы сдлеали заранее. 

# dataframe_concatedTone_train, dataframe_concatedTone_test = train_test_split(dataframe_concatedTone, 
#                                                                              test_size = 0.2, 
#                                                                              random_state = 42)

In [226]:
dataframe_concatedTone_train = pd.concat([dataframeTrainAug, dataframeTone_train])
dataframe_concatedTone_train = dataframe_concatedTone_train.drop_duplicates(subset=['text'])

In [227]:
text_transformer = TfidfVectorizer(
                                   ngram_range=(1, 4), 
                                   lowercase=True, 
                                   max_features=150000)

In [228]:
%%time
Xaug_train = text_transformer.fit_transform(dataframe_concatedTone_train['text'])
Xaug_test = text_transformer.transform(dataframeTone_test['text']) # Как для прошлой модели
yaug_train = dataframe_concatedTone_train['tone']
yaug_test = dataframeTone_test['tone'] # Как для прошлой модели

CPU times: user 2.08 s, sys: 103 ms, total: 2.18 s
Wall time: 2.28 s


In [229]:
LogReg = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)
LogReg.fit(Xaug_train, yaug_train)
yuag_pred = LogReg.predict(Xaug_test)

In [230]:
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(yaug_test, yuag_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.85      0.90      0.88       496
     class 1       0.89      0.83      0.85       403
     class 2       0.87      0.84      0.85       146

    accuracy                           0.87      1045
   macro avg       0.87      0.86      0.86      1045
weighted avg       0.87      0.87      0.86      1045



Сравним с предыдущим скором, как мы можем видеть качество модели по некоторым п
унктам выросло, но очень не сильно

In [231]:
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.85      0.92      0.88       496
     class 1       0.89      0.82      0.85       403
     class 2       0.88      0.79      0.83       146

    accuracy                           0.87      1045
   macro avg       0.87      0.85      0.86      1045
weighted avg       0.87      0.87      0.86      1045



Таким образом, модель стала работать лучше, хотя возможно, она переобучилась. Далее предстоит провести кросс валидацию, попробовать изменять гипер-параметры и стратегию аугментации(брать не топ 1 по близости(например)).
Полезные ссылки:
1. https://www.kaggle.com/theoviel/using-word-embeddings-for-data-augmentation
2. http://www.machinelearning.ru/wiki/images/b/b3/Word2Vec.pdf
3. https://github.com/akutuzov/webvectors
4. https://universaldependencies.org/u/pos/

СДЕЛАНО: Возможно, она будет работаь еще хуже, если агументировать отдельно и тестовую выборку и трейн выборки и уже сравнивать метрики по одним и тем же выборкам

svc = SVC()
lsvc = LinearSVC(random_state=123)
rforest = RandomForestClassifier(random_state=123)
dtree = DecisionTreeClassifier()


Возможно, высокий скор из-за того, что я не использовал ту же тестовую выборку, то я считаю. 

Поресечрчить зависимость скора от аугментации мусорных слов и наоборот с eli5.

Поменять solver

### Кросс-валидация 

In [232]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [233]:
%%time
cv_results = cross_val_score(LogReg, Xaug_train, yaug_train, cv=skf, scoring='f1_micro')

CPU times: user 130 ms, sys: 103 ms, total: 233 ms
Wall time: 44.5 s


Приятно видеть, что кросс-валидация более или менее стабильна на всех фолдах

In [234]:
cv_results, cv_results.mean()

(array([0.89143546, 0.88661037, 0.8986731 , 0.88292094, 0.89438745]),
 0.8908054654268582)

### Параметризация аугментации

Теперь попробуем вычислить скор, если аугментировать не все подряд, а только интересующие тэги: наречия и прилагательные 

In [235]:
mystem2uposMod = {'A': ' ADJ',
 'ADV': 'ADV',
 'ADVPRO': 'STOP',
 'ANUM': 'STOP',
 'APRO': 'STOP',
 'COM': 'STOP',
 'CONJ': 'STOP',
 'INTJ': 'STOP',
 'NONLEX': 'STOP',
 'NUM': 'STOP',
 'PART': 'STOP',
 'PR': 'STOP',
 'S': 'STOP',
 'SPRO': 'STOP',
 'UNKN': 'X',
 'V': 'STOP'}

Скопируем все незакоментированное для аугментированной модели 

In [236]:
dataframeTrainAug = dataframeTone_train.copy()
dataframeTrainAug.text = dataframeTrainAug.text.apply(lambda x: tag_mystem(x, mapping = mystem2uposMod))

dataframeTrainAug['text'] = replacement_by_WE(dataframeTrainAug.text) 
dataframeTrainAug.to_csv("/Users/magomednikolaev/Documents/Работа/NLP/we/comments_train_augmeted_aadv_tags.xlsx",) 
dataframe_concatedTone_train = pd.concat([dataframeTrainAug, dataframeTone_train])
dataframe_concatedTone_train = dataframe_concatedTone_train.drop_duplicates(subset=['text'])

Xaug_aadj_train = text_transformer.fit_transform(dataframe_concatedTone_train['text'])
Xaug_aadj_test = text_transformer.transform(dataframeTone_test['text']) # Как для прошлой модели
yaug_aadj_train = dataframe_concatedTone_train['tone']
yaug_aadj_test = dataframeTone_test['tone'] # Как для прошлой модели

LogReg = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)
LogReg.fit(Xaug_aadj_train, yaug_aadj_train)
yuag_aadj_pred = LogReg.predict(Xaug_aadj_test)

100%|██████████| 4176/4176 [02:20<00:00, 29.66it/s]


Без аугментации

In [237]:
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(yaug_test, yuag_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.85      0.90      0.88       496
     class 1       0.89      0.83      0.85       403
     class 2       0.87      0.84      0.85       146

    accuracy                           0.87      1045
   macro avg       0.87      0.86      0.86      1045
weighted avg       0.87      0.87      0.86      1045



Аугментация по всем тэгам

In [238]:
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.85      0.92      0.88       496
     class 1       0.89      0.82      0.85       403
     class 2       0.88      0.79      0.83       146

    accuracy                           0.87      1045
   macro avg       0.87      0.85      0.86      1045
weighted avg       0.87      0.87      0.86      1045



Аугментация по прилагательным и наречиям

In [239]:
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(yaug_aadj_test, yuag_aadj_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.86      0.91      0.88       496
     class 1       0.88      0.84      0.86       403
     class 2       0.88      0.83      0.85       146

    accuracy                           0.87      1045
   macro avg       0.87      0.86      0.86      1045
weighted avg       0.87      0.87      0.87      1045



/Users/magomednikolaev/opt/anaconda3/envs/nlp/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/magomednikolaev/opt/anaconda3/envs/nlp/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/